## Calibration tests
### Pose2Sim calibration.py functions
### Data projet Cobotique

Auteurs : Léa Drolet-Roy

Création : 2025-02-18

Dernière modification :

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

import pyswarms as ps
from pyswarms.utils.plotters import plot_cost_history

from Calibration import calibration as cali

np.set_printoptions(precision=4, suppress=True)

### Constants

In [2]:
CHECKERBOARD = (7,4)
SQUARE_SIZE = 48

cams = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
time_threshold = 15

path = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique'
u101 = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\u101'
u102 = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\u102'

extrinsics_corners_nb = CHECKERBOARD
extrinsics_square_size = SQUARE_SIZE

objpoints = np.zeros((extrinsics_corners_nb[0] * extrinsics_corners_nb[1], 3), np.float32)
objpoints[:, :2] = np.mgrid[0:extrinsics_corners_nb[0], 0:extrinsics_corners_nb[1]].T.reshape(-1, 2)
objpoints[:, :2] = objpoints[:, 0:2] * extrinsics_square_size

cal = cali(CHECKERBOARD, SQUARE_SIZE)

In [ ]:
# To find images where we see the checkerboard, for individual cameras
def save_images_checkerboard(path, dir, cam):
    for fname in os.listdir(os.path.join(dir, cam)):
        savepath = os.path.join(path, 'calibration', 'intrinsics', cam)
        os.makedirs(savepath, exist_ok=True)
        savename = dir[-4:] + '_' + cam + fname
        # if savename not in os.listdir(os.path.join(path, 'calibration', 'intrinsics', cam)):
        ret, img, _ = cal.findCorners(os.path.join(dir, cam, fname), subpix = False)

        if ret == True:
            cv2.imwrite(os.path.join(savepath, savename), img)
        
    return

In [ ]:
# for cam in os.listdir(u101):
#     save_images_checkerboard(path, u101, cam)
    
# for cam in os.listdir(u102):
#     save_images_checkerboard(path, u102, cam)

In [3]:
intrinsics_config_dict = {'calculate_intrinsics':True, 'overwrite_intrinsics':True, 'show_detection_intrinsics':False, 'intrinsics_method':'board',
                          'intrinsics_extension': 'png', 'intrinsics_corners_nb':CHECKERBOARD, 'intrinsics_square_size':SQUARE_SIZE,
                          'board': {'intrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'intrinsics_corners_nb': CHECKERBOARD, 'intrinsics_square_size':SQUARE_SIZE}}

extrinsics_config_dict = {'calculate_extrinsics':True, 'show_detection_extrinsics':False, 'extrinsics_method':'board',
                          'extrinsics_extension': 'png', 'extrinsics_corners_nb':CHECKERBOARD, 'extrinsics_square_size':SQUARE_SIZE,
                          'board': {'extrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'extrinsics_corners_nb': CHECKERBOARD, 'extrinsics_square_size':SQUARE_SIZE}}

calib_dir = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed"
image_size = (1280, 720)

# ret, C, S, D, K, R, T = calibration.calibrate_intrinsics(calib_dir, intrinsics_config_dict)
# ret, C, S, D, K, R, T = cal.calibrate_intrinsics(calib_dir, image_size, camera_matrix)
# print(f'reproj error (mm): {ret} \nCam: {C} \nImage Size: {S} \nIntrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')


In [ ]:
K = np.array(K)
np.savez('Intrinsics_K_final.npz', K)

D = np.array(D)
np.savez('Intrinsics_D_final.npz', D)

In [4]:
K = np.load('saved_cobotique/Intrinsics_K_final.npz')['arr_0']
D = np.load('saved_cobotique/Intrinsics_D_final.npz')['arr_0']

print(f'Intrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')

Intrinsics: 
[[648.4489   0.     648.7351]
 [  0.     649.7999 362.77  ]
 [  0.       0.       1.    ]] 
[[655.7549   0.     657.0469]
 [  0.     657.8988 357.1172]
 [  0.       0.       1.    ]] 
[[653.0931   0.     643.5985]
 [  0.     653.7393 362.3068]
 [  0.       0.       1.    ]] 
[[641.12     0.     659.4241]
 [  0.     637.8964 368.091 ]
 [  0.       0.       1.    ]] 
[[645.08     0.     652.0756]
 [  0.     646.7413 361.2488]
 [  0.       0.       1.    ]] 
[[649.8124   0.     657.3696]
 [  0.     650.0296 366.7623]
 [  0.       0.       1.    ]]


In [5]:
angle_threshold = 60 #degrees
coverage_threshold = (0.01, 0.99)
image_size = (1280, 720)

for i, cam in enumerate(cams):
    area = cal.refine_image_selection(calib_dir, 'intrinsics_refine', cam, K[i], D[i], image_size, angle_threshold, coverage_threshold)
    print(area)

0.33992838541666665
0.8218532986111111
0.8710416666666667
0.29476128472222224
0.7610709635416667
0.7409201388888889


In [5]:
path_stereo = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\stereo_confirmed'
path_intrinsics = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed'

# cal.find_matching_images(path, 'u101', cams, time_threshold, path_stereo, path_intrinsics)
# cal.find_matching_images(path, 'u102', cams, time_threshold, path_stereo, path_intrinsics)

## Stereo calibration with matching images for each pair of cameras

### Loop on possible camera pairs

In [ ]:
cams = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
for camL in cams:
    for camR in cams[cams.index(camL)+1:]:
        print(camL, camR)
        stereopath = os.path.join(path_stereo, f'{camL}_{camR}')
        Left_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[2] == camL]
        Right_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[2] == camR]
        
        if len(Left_Paths) >= 1 and len(Right_Paths) >= 1:
            Left_imgpoints, Left_Paths, Right_imgpoints, Right_Paths = cal.GenerateImagepoints(Left_Paths, Right_Paths, CHECKERBOARD)
            if len(Left_imgpoints) >= 1 and len(Right_imgpoints) >= 1:

                try:
                    Ki = K[cams.index(camL)]
                    Di = D[cams.index(camL)]
                    Left_Params = cal.CalibrateCamera(Left_Paths, Left_imgpoints, K[cams.index(camL)], D[cams.index(camL)])

                except IndexError:
                    Left_Params = cal.CalibrateCamera(Left_Paths, Left_imgpoints)

                try:
                    Kj = K[cams.index(camR)]
                    Dj = D[cams.index(camR)]
                    Right_Params = cal.CalibrateCamera(Right_Paths, Right_imgpoints, Kj, Dj)
                except IndexError:
                    Right_Params = cal.CalibrateCamera(Right_Paths, Right_imgpoints)
                Left_MeanError = np.mean(np.array(Left_Params['Errors']), axis=-1)
                Right_MeanError = np.mean(np.array(Right_Params['Errors']), axis=-1)

                Left_Params['Imgpoints'] = Left_imgpoints
                Left_Params['MeanError'] = Left_MeanError

                Right_Params['Imgpoints'] = Right_imgpoints
                Right_Params['MeanError'] = Right_MeanError

                print('Reprojection Error Left:  {:.4f}'.format(Left_MeanError))
                print('Reprojection Error Right: {:.4f}'.format(Right_MeanError))                

                Stereo_Params = cal.StereoCalibration(Left_Params, Right_Params, Left_imgpoints, Right_imgpoints, Left_Paths)
                print('Transformation Matrix:')
                print(Stereo_Params['Transformation'])

                cal.SaveParameters(camL, camR, Stereo_Params, Left_Params, Right_Params)
            else:
                print('Not enough images', '\n')
        else:
            print('Not enough images', '\n')

c1 c2
Not enough images 

c1 c3
Not enough images 

c1 c4
Reprojection Error Left:  0.7715
Reprojection Error Right: 1.3015
Transformation Matrix:
[[   0.4834    0.4534   -0.7488  878.467 ]
 [  -0.3757    0.8801    0.2903 -165.104 ]
 [   0.7907    0.1409    0.5958   32.3176]
 [   0.        0.        0.        1.    ]]
c1 c5
Reprojection Error Left:  0.8343
Reprojection Error Right: 0.7774
Transformation Matrix:
[[    0.351     -0.4999     0.7917 -1836.7874]
 [    0.4327     0.8365     0.3363  -809.4149]
 [   -0.8304     0.2245     0.5099  1044.6748]
 [    0.         0.         0.         1.    ]]
c1 c6
Reprojection Error Left:  0.7373
Reprojection Error Right: 0.8058
Transformation Matrix:
[[   0.8841   -0.1576   -0.4399 -109.2927]
 [   0.0675    0.9746   -0.2135  262.2687]
 [   0.4624    0.1591    0.8723  148.2321]
 [   0.        0.        0.        1.    ]]
c2 c3
Reprojection Error Left:  1.5921
Reprojection Error Right: 1.2066
Transformation Matrix:
[[  0.7488   0.2599  -0.6097 628.

## Manual selection of reference points on tabletop

In [6]:
def mouse_callback(event, x, y, flags, param):
    """Callback function to capture mouse clicks."""
    global clicked_points, cam
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_points.append((x, y))
        point_num = len(clicked_points)

        # Draw a small circle at the clicked point
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        # Add text indicating the point number
        cv2.putText(image, f"{point_num}", (x, y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        # Refresh the display
        cv2.imshow("Image", image)

        # Stop after collecting 3 points
        if len(clicked_points) == 3:
            print(f"Selected points for {cam}:", clicked_points)
            image_points[cam] = clicked_points.copy()
            cv2.destroyAllWindows()
    
    return clicked_points

In [ ]:
# Select 1 image per camera view where we see the reference points
img1 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c1\u102_c1g1_p3_1.png"
img4 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c4\u102_c4g1_p3_338.png"
img5 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c5\u102_c5g1_p3_399.png"
img6 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c6\u102_c6g1_p3_404.png"

instructions = f"Point 1 : origin 3 from L/4 from R \n Point 2: x-axis -> 2 Middle \n Point 3: y-axis 2 from R"

# Dictionary to store points for each image
image_points = {}

for img_path, cam in zip([img1, img4, img5, img6], ['cam1', 'cam4', 'cam5', 'cam6']):
    image = cv2.imread(img_path)
    clicked_points = []
    # Display instructions on the image
    cv2.putText(image, instructions, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Image", image)
    cv2.setMouseCallback("Image", mouse_callback)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save points to a JSON file
with open("clicked_points_2.json", "w") as file:
    json.dump(image_points, file, indent=4)

Selected points for cam1: [(831, 618), (936, 711), (913, 576)]
Selected points for cam4: [(694, 680), (827, 671), (679, 592)]
Selected points for cam5: [(361, 538), (229, 601), (433, 606)]
Selected points for cam6: [(478, 566), (552, 643), (552, 521)]


In [ ]:
# Select 1 image per camera view where we see the reference points
img2 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c2\u102_c2g1_p3_396.png"
img3 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c3\u102_c3g1_p3_391.png"

instructions = f"Point 1 : origin 3 from L/4 from R \n Point 2: x-axis -> 2 Middle \n Point 3: y-axis 2 from R"

# Dictionary to store points for each image
image_points = {}

for img_path, cam in zip([img2, img3], ['cam2', 'cam3']):
    image = cv2.imread(img_path)
    clicked_points = []
    # Display instructions on the image
    cv2.putText(image, instructions, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Image", image)
    cv2.setMouseCallback("Image", mouse_callback)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save points to a JSON file
with open("saved_cobotique/clicked_points.json", "r") as file:
    image_points_base = json.load(file)

image_points_base.update(image_points)

with open("saved_cobotique/clicked_points_2.json", "w") as file:
    json.dump(image_points_base, file, indent=4)

Selected points for cam2: [(692, 651), (574, 647), (688, 703)]
Selected points for cam3: [(605, 657), (509, 714), (633, 690)]


In [7]:
# Load points from JSON file
with open("saved_cobotique/clicked_points_2.json", "r") as file:
    image_points = json.load(file)

image_points_adjusted = image_points.copy()

# Adjust points for cameras 2 and 3 (extend y-axis)
for cam, (p0, p1, p2) in image_points_adjusted.items():
    if cam in ['cam2', 'cam3']:
        x0, y0 = p0
        x2, y2 = p2
        
        dx, dy = x2 - x0, y2 - y0
        p3 = (x2 + dx, y2 + dy) # Extend in the forward direction
        
        image_points_adjusted[cam][2] = p3

with open("saved_cobotique/clicked_points_adjusted.json", "w") as file:
    json.dump(image_points_adjusted, file, indent=4)

img1 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c1\u102_c1g1_p3_1.png"
img2 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c2\u102_c2g1_p3_396.png"
img3 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c3\u102_c3g1_p3_391.png"
img4 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c4\u102_c4g1_p3_338.png"
img5 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c5\u102_c5g1_p3_399.png"
img6 = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed\c6\u102_c6g1_p3_404.png"

img_paths = [img1, img2, img3, img4, img5, img6]
cams = ['cam1', 'cam2', 'cam3', 'cam4', 'cam5', 'cam6']

In [ ]:
# Visualize selected points (for validation purpose)

with open("saved_cobotique/clicked_points_adjusted.json", "r") as file:
    image_points = json.load(file)

fig, axs = plt.subplots(2, 3, figsize=(16,12))
axs = axs.flatten()

for i in range(6):
    c1 = int(cams[i][-1]) - 1
        
    pts1_im = np.array(image_points[cams[i]], dtype=np.float32)
    img_path1 = img_paths[i]
    img = cv2.imread(img_path1)

    img = draw_circles(img, pts1_im, None, False)
    axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[i].axis('off')
    axs[i].set_title(f'Cam {i+1}', fontsize=8)

fig.suptitle("Original 2D Points (Blue)")
fig.tight_layout()
plt.show()
plt.close()


In [8]:
c1_c4 = np.load('saved_cobotique/c1_c4_parameters.npz')['Transformation']
c1_c5 = np.load('saved_cobotique/c1_c5_parameters.npz')['Transformation']
c1_c6 = np.load('saved_cobotique/c1_c6_parameters.npz')['Transformation']
c2_c3 = np.load('saved_cobotique/c2_c3_parameters.npz')['Transformation']
c2_c5 = np.load('saved_cobotique/c2_c5_parameters.npz')['Transformation']
c3_c5 = np.load('saved_cobotique/c3_c5_parameters.npz')['Transformation']
c4_c6 = np.load('saved_cobotique/c4_c6_parameters.npz')['Transformation']

In [9]:
def var_name(var, scope=globals()):
    return [name for name, value in scope.items() if value is var][0]

def compare_transfo(T12, T23, T13):
    print(f'Original {var_name(T13)}')
    print(T13)
    T13_calc = T12 @ T23
    print(f'Calculated {var_name(T13)}')
    print(T13_calc)
    print('Calculated I')
    print(np.linalg.inv(T13_calc) @ T13, '\n')

    return

In [10]:
compare_transfo(np.linalg.inv(c1_c4), c1_c6, c4_c6) # OK

compare_transfo(np.linalg.inv(c2_c3), c2_c5, c3_c5) # OK

Original c4_c6
[[   0.6799   -0.552     0.4827 -870.0134]
 [   0.578     0.8085    0.1104 -208.2455]
 [  -0.4512    0.204     0.8688  497.4683]
 [   0.        0.        0.        1.    ]]
Calculated c4_c6
[[   0.7677   -0.3165    0.5572 -546.4253]
 [   0.5254    0.8087   -0.2644  -55.3691]
 [  -0.3669    0.4958    0.7871  932.795 ]
 [   0.        0.        0.        1.    ]]
Calculated I
[[   0.9912   -0.0737    0.1098 -168.9978]
 [   0.0285    0.9297    0.3672 -237.029 ]
 [  -0.1291   -0.3608    0.9236 -482.5419]
 [   0.        0.        0.        1.    ]] 

Original c3_c5
[[   0.9378   -0.0976   -0.3332   96.2663]
 [   0.0835    0.9949   -0.0564 -299.5696]
 [   0.337     0.0251    0.9412 -157.1114]
 [   0.        0.        0.        1.    ]]
Calculated c3_c5
[[   0.9514   -0.0412   -0.3051    1.8994]
 [  -0.0133    0.9846   -0.1744 -292.8513]
 [   0.3076    0.17      0.9362   58.388 ]
 [   0.        0.        0.        1.    ]]
Calculated I
[[   0.9948   -0.0983   -0.0267   23.5894]


## Particle Swarm Optimizer pour paramètres intrinsèques

par image et par caméra

In [6]:
path = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics\c1"

objpoints = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objpoints[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objpoints[:, :2] = objpoints[:, 0:2] * SQUARE_SIZE

im_name = os.path.join(path, os.listdir(path)[30])
img = cv2.imread(os.path.join(path, im_name))
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
_, imgpoints = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
imgpoints = imgpoints.squeeze()
imgpoints = imgpoints.astype(np.float32)

_, r, t = cv2.solvePnP(objpoints, imgpoints, K[0], D[0])
r = r.flatten()
t = t.flatten()

# Define the initial parameter values
params_c1 = K[0]
init_params = [params_c1[0,0], params_c1[1,1], params_c1[0,2], params_c1[1,2]]
init_params += list(D[0])
init_params += [r[0], r[1], r[2]]
init_params += [t[0], t[1], t[2]]

init_params = np.array(init_params, dtype=np.float32)  # Convert to NumPy array
print("Initial Parameters:", init_params)

# Define Optimizer hyperparameters
n_particles = 50  # Number of particles

init_pos = np.tile(init_params, (n_particles, 1))  # Shape: (30, 15)
print("Initial Position Shape:", init_pos.shape)  # Should be (30, 15)

param_bounds = ([500, 500, 600, 300, -0.1, -0.1, -0.01, -0.01, -0.1, -np.pi, -np.pi, -np.pi, -1000, -1000, -1000],  # Lower bounds
                [700, 700, 700, 400, 0.1, 0.1, 0.01, 0.01, 0.1, np.pi, np.pi, np.pi, 1000, 1000, 2000])          # Upper bounds
param_bounds = np.array(param_bounds, dtype=np.float32).reshape((2,15))
print(param_bounds)
print('Param bounds Shape:', param_bounds.shape)

options = {'c1': 1.5, 'c2': 0.3, 'w': 0.5}  # Cognitive, social, and inertia weights

# Define projection Model
def project_points(params, obj_pts):
    """
    Projects 3D points to 2D using intrinsic parameters.
    params = [fx, fy, cx, cy, k1, k2, p1, p2, k3] (9 parameters)
    """
    fx, fy, cx, cy, k1, k2, p1, p2, k3, r1, r2, r3, t1, t2, t3 = params
    
    camera_matrix = np.array([[fx, 0, cx],
                              [0, fy, cy],
                              [0,  0,  1] ], dtype=np.float32)
    dist_coeffs = np.array([k1, k2, p1, p2, k3])

    rvec = np.array([r1, r2, r3])
    tvec = np.array([t1, t2, t3])
    
    # Project points
    projected_pts, _ = cv2.projectPoints(obj_pts, rvec, tvec, camera_matrix, dist_coeffs)
    return projected_pts.squeeze()

# Define cost function
def RMSE(params):
    errors = np.zeros(n_particles)  # Array to store errors

    for i in range(n_particles):  
        imgp_proj = project_points(params[i], objpoints)  # Project points for this particle
        
        squared_diff = (imgpoints - imgp_proj) ** 2
        mse = np.mean(np.sum(squared_diff, axis=1))
        rmse = np.sqrt(mse)
        errors[i] = np.mean(rmse)  # Store mean RMSE for this particle
    
    return errors  # Return array of errors (one per particle)

# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=15, options=options, bounds=param_bounds, init_pos=init_pos)
best_error, best_params = optimizer.optimize(RMSE, iters=500)

plot_cost_history(cost_history=optimizer.cost_history)
plt.show()

# Extract optimized parameters
fx, fy, cx, cy, k1, k2, p1, p2, k3, r1, r2, r3, t1, t2, t3 = best_params

# Create final camera matrix
optimized_camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
optimized_dist_coeffs = np.array([k1, k2, p1, p2, k3])
optimized_rvec = np.array([r1, r2, r3])
optimized_tvec = np.array([t1, t2, t3])

print("Optimized Camera Matrix:\n", optimized_camera_matrix)
print("Optimized Distortion Coefficients:\n", optimized_dist_coeffs)
print("Optimized Rotation Vector:\n", optimized_rvec)
print("Optimized Translation Vector:\n", optimized_tvec)
print(f"Best Reprojection Error: {best_error}")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.

Pour toutes les images de toutes les caméras en même temps, (r,t) avec SolvePnP pour chaque image

In [ ]:
# Constants
num_cameras = 6
num_params_per_cam = 9
total_params = num_cameras * num_params_per_cam
n_particles = 50  # Number of particles
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.5}  # PSO Hyperparameters

# Paths to calibration images for each camera
base_path = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics_refine"
camera_paths = [os.path.join(base_path, f'c{i+1}') for i in range(num_cameras)]

# Generate 3D object points (same for all images)
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * SQUARE_SIZE

# Initial parameters for each camera
K_fab = [[[644, 0, 656], [0, 643, 358], [0,0,1]],
         [[638, 0, 645], [0, 637, 363], [0,0,1]],
         [[645, 0, 645], [0, 643, 362], [0,0,1]],
         [[644, 0, 637], [0, 643, 358], [0,0,1]],
         [[642, 0, 650], [0, 641, 357], [0,0,1]],
         [[644, 0, 650], [0, 642, 362], [0,0,1]]]
K_fab = np.array(K_fab, dtype=np.float32)

K_calc = np.load('Intrinsics_K.npz')['arr_0']
D_calc = np.load('Intrinsics_D.npz')['arr_0']

K = K_fab       # Choose between theorical or calculated params K_fab / K_calc
D = [np.zeros(5, dtype=np.float32) for _ in range(num_cameras)]  # Distortion Coeffs

# Store object points and image points for all cameras
objpoints_list = []  # 3D points (same for all cameras)
imgpoints_list = [[] for _ in range(num_cameras)]  # 2D points per camera
extrinsics_list = [[] for _ in range(num_cameras)] 

for cam_idx, cam_path in enumerate(camera_paths):
    img_files = os.listdir(cam_path)
    for img_name in img_files:
        img_path = os.path.join(cam_path, img_name)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
        if ret:
            corners = corners.squeeze().astype(np.float32)
            imgpoints_list[cam_idx].append(corners)
            objpoints_list.append(objp)

init_params = []
for cam_idx in range(num_cameras):
    cam_init = [K[cam_idx,0, 0], K[cam_idx,1, 1], K[cam_idx,0, 2], K[cam_idx,1, 2]]
    cam_init += list(D[cam_idx])
    init_params.extend(cam_init)

init_params = np.array(init_params, dtype=np.float32)

# Define parameter bounds
lower_bounds = [500, 500, 600, 300, -0.1, -0.1, -0.01, -0.01, -0.1]
upper_bounds = [700, 700, 700, 400, 0.1, 0.1, 0.01, 0.01, 0.1]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))

In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.5}  # PSO Hyperparameters

# Projection function
def project_points(camera_matrix, dist_coeffs, r, t, obj_pts):
    projected_pts, _ = cv2.projectPoints(obj_pts, r, t, camera_matrix, dist_coeffs)
    return projected_pts.squeeze()

# Define cost function
def RMSE(params):
    errors = np.zeros(n_particles)
    num_images = len(objpoints_list)  # Total images used for calibration
    
    for i in range(n_particles):
        total_error = 0
        for cam_idx in range(num_cameras):
            cam_params = params[i][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            fx, fy, cx, cy, k1, k2, p1, p2, k3 = cam_params
            camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0,  0,  1]], dtype=np.float64)
            dist_coeffs = np.array([k1, k2, p1, p2, k3], dtype=np.float64)

            for (obj_pts, img_pts) in zip(objpoints_list, imgpoints_list[cam_idx]):
                _, r, t = cv2.solvePnP(obj_pts, img_pts, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_EPNP)

                projected_pts = project_points(camera_matrix, dist_coeffs, r, t, obj_pts)

                squared_diff = (img_pts - projected_pts) ** 2
                mse = np.mean(np.sum(squared_diff, axis=1))
                rmse = np.sqrt(mse)
                total_error += rmse
        
        errors[i] = total_error / (num_cameras * num_images)
    
    return errors[i]

# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=total_params, options=options, bounds=param_bounds)
best_error, best_params = optimizer.optimize(RMSE, iters=20)

# Extract optimized parameters for each camera
optimized_params = np.split(best_params, num_cameras)
for cam_idx in range(num_cameras):
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = optimized_params[cam_idx]
    optimized_camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    optimized_dist_coeffs = np.array([k1, k2, p1, p2, k3])
    print(f"Camera {cam_idx+1} Optimized Parameters:")
    print("Optimized Camera Matrix:\n", optimized_camera_matrix)
    print("Optimized Distortion Coefficients:\n", optimized_dist_coeffs)
    print("\n")

print(f"Best Reprojection Error: {best_error}")

plot_cost_history(cost_history=optimizer.cost_history)
plt.show()


2025-02-28 09:42:44,303 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.5}
pyswarms.single.global_best:  10%|█         |2/20, best_cost=1.46C:\Users\LEA\AppData\Local\Temp\ipykernel_76732\2601850050.py:31: RuntimeWarning: overflow encountered in square
  squared_diff = (img_pts - projected_pts) ** 2
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=1.34
2025-02-28 09:44:36,865 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.3413528294807702, best pos: [629.0066 692.2739 609.1485 360.4797   0.0982  -0.0352   0.0028  -0.0043
  -0.0398 628.8125 615.7793 620.3899 378.6241  -0.0332  -0.0843   0.0002
  -0.0009  -0.0363 626.9632 587.0663 642.742  354.7231  -0.0028  -0.0474
  -0.0046   0.0006   0.024  578.8816 560.0724 619.7809 314.7627  -0.0416
   0.0479   0.0067   0.0004  -0.012  658.3184 525.7467 655.8684 359.267
   0.0647   0.0153   0.0063   0.0053   0.0202 512.6711 629.2437 674.0313
 306.5824  -0.0234 

Camera 1 Optimized Parameters:
Optimized Camera Matrix:
 [[629.0066   0.     609.1485]
 [  0.     692.2739 360.4797]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [ 0.0982 -0.0352  0.0028 -0.0043 -0.0398]


Camera 2 Optimized Parameters:
Optimized Camera Matrix:
 [[628.8125   0.     620.3899]
 [  0.     615.7793 378.6241]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0332 -0.0843  0.0002 -0.0009 -0.0363]


Camera 3 Optimized Parameters:
Optimized Camera Matrix:
 [[626.9632   0.     642.742 ]
 [  0.     587.0663 354.7231]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0028 -0.0474 -0.0046  0.0006  0.024 ]


Camera 4 Optimized Parameters:
Optimized Camera Matrix:
 [[578.8816   0.     619.7809]
 [  0.     560.0724 314.7627]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0416  0.0479  0.0067  0.0004 -0.012 ]


Camera 5 Optimized Parameters:
Optimized Camera Matrix:
 [[658.3184   0.     655.8684]
 

## 3D reprojection error

In [24]:
# Function to project 3D points to 2D
def project_points(points_3d, projMat, K, D):
    rvec, _ = cv2.Rodrigues(projMat[0:3,0:3])
    projected_2d, _ = cv2.projectPoints(points_3d, rvec, projMat[0:3,3], K, D)
    return projected_2d.squeeze()

# Compute RMSE
def compute_rmse(original_pts, points_3d, projMat, K, D):
    projected_pts = project_points(points_3d, projMat, K, D)
    error = np.linalg.norm(original_pts - projected_pts, axis=1)  # Euclidean distance per point
    rmse = np.sqrt(np.mean(error**2))  # Compute RMSE
    return rmse

## Stereo initial extrinsic parameters

In [16]:
# Create array of all stereo images with 'im_name, img_pts, cam_ timestamp'
stereo_images = {'Name':[], 'Camera':[], 'Img_pts':[], 'Time':[]}

for user in ['u101', 'u102']:

    for cam1 in cams:
        cali1 = cal.retrieve_cali_info(path, user, cam1[-1])
        for cam2 in cams[cams.index(cam1)+1:]:
            cali2 = cal.retrieve_cali_info(path, user, cam2[-1])
            path_stereo = os.path.join(path_stereo, f'{cam1}_{cam2}')

            for i,t1 in enumerate(cali1[1,:]):
                t1 = float(t1)
                for j,t2 in enumerate(cali2[1,:]):
                    t2 = float(t2)
                    if abs(t1-t2) < time_threshold:

                        name1 = user + '_' + cam1 + cali1[0,i]
                        name2 = user + '_' + cam2 + cali2[0,j]
                        
                        im1In = f'{i}_{j}_{cam1}_{name1}' in list(os.listdir(path_stereo))
                        im2In = f'{i}_{j}_{cam2}_{name2}' in list(os.listdir(path_stereo))
                                
                        if im1In and im2In:
                            Lret, Limg, Limg_pts = cal.findCorners(os.path.join(path_stereo, f'{i}_{j}_{cam1}_{name1}'))
                            Rret, Rimg, Rimg_pts = cal.findCorners(os.path.join(path_stereo, f'{i}_{j}_{cam2}_{name2}'))

                            if Lret and Rret:
                                stereo_images['Name'].append(name1)
                                stereo_images['Camera'].append(int(cam1[-1]))
                                stereo_images['Img_pts'].append(Limg_pts)
                                stereo_images['Time'].append(t1)

                                stereo_images['Name'].append(name2)
                                stereo_images['Camera'].append(int(cam2[-1]))
                                stereo_images['Img_pts'].append(Rimg_pts)
                                stereo_images['Time'].append(t2)

with open('stereo_data_final.pkl', 'wb') as f:
    pickle.dump(stereo_images, f)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:\\Users\\LEA\\Desktop\\Poly\\Trampo\\data_cobotique\\calibration_0703\\stereo_confirmed\\cam1_cam2'

In [12]:
with open('saved_cobotique/stereo_data_final.pkl', 'rb') as f:
    stereo_data = pickle.load(f)

In [13]:
Tcam1 = np.eye((4))
Tcam6 = c1_c6
Tcam4 = c1_c4
Tcam5 = c1_c5
Tcam3 = Tcam5 @ np.linalg.inv(c3_c5)
Tcam2 = Tcam5 @ np.linalg.inv(c2_c5)

projMat = [Tcam1, Tcam2, Tcam3, Tcam4, Tcam5, Tcam6]

for mat in projMat:
    print(mat)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[   -0.5464    -0.5674     0.6161 -2024.6351]
 [    0.043      0.7156     0.6972 -1010.4567]
 [   -0.8364     0.4074    -0.3665  1777.0763]
 [    0.         0.         0.         1.    ]]
[[    0.1142    -0.5128     0.8509 -1867.7053]
 [    0.2121     0.8493     0.4834  -499.4606]
 [   -0.9705     0.1253     0.2058  1207.9708]
 [    0.         0.         0.         1.    ]]
[[   0.4834    0.4534   -0.7488  878.467 ]
 [  -0.3757    0.8801    0.2903 -165.104 ]
 [   0.7907    0.1409    0.5958   32.3176]
 [   0.        0.        0.        1.    ]]
[[    0.351     -0.4999     0.7917 -1836.7874]
 [    0.4327     0.8365     0.3363  -809.4149]
 [   -0.8304     0.2245     0.5099  1044.6748]
 [    0.         0.         0.         1.    ]]
[[   0.8841   -0.1576   -0.4399 -109.2927]
 [   0.0675    0.9746   -0.2135  262.2687]
 [   0.4624    0.1591    0.8723  148.2321]
 [   0.        0.        0.        1.    ]]


In [ ]:
def visualize_projection(img_path, pts_2d_original, pts_3d, projMat, K, D):
    """
    Visualize original 2D points and projected 3D points on an image.

    Parameters:
    - img_path: Path to the image.
    - pts_2d_original: Original 2D keypoints (Nx2).
    - pts_3d: Triangulated 3D points (Nx3).
    - projMat: Projection matrix (3x4).
    - K: Camera intrinsic matrix (3x3).
    - D: Distortion coefficients (1x5 or 1x4).
    """

    # Display the image
    plt.figure(figsize=(10, 6))

    for i in range(len(img_path)):

        # Load the image
        img = cv2.imread(img_path[i])
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path[i]}")

        # Project 3D points onto the image plane
        pts_2d_projected, _ = cv2.projectPoints(pts_3d[i], projMat[i][0:3,0:3], projMat[i][0:3,3], K[i], D[i])

        # Reshape projected points
        pts_2d_projected = pts_2d_projected.squeeze()

        # Draw the original and projected points
        for j in range(len(pts_2d_original[i])):
            x_orig, y_orig = map(int, pts_2d_original[i][j])  # Original 2D point
            x_proj, y_proj = map(int, pts_2d_projected[j])  # Reprojected 3D point

            # Draw original points in BLUE
            cv2.circle(img, (x_orig, y_orig), 4, (255, 0, 0), -1)

            # Draw reprojected points in GREEN
            cv2.circle(img, (x_proj, y_proj), 4, (0, 255, 0), -1)

            # Draw line between them (error visualization)
            cv2.line(img, (x_orig, y_orig), (x_proj, y_proj), (0, 255, 255), 1)

            fig = plt.subplot(1,2,i+1)
            fig.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    plt.title("Original 2D Points (Blue) vs. Projected 3D Points (Green)")
    plt.show()

## PSO for extrinsic parameters (with Bundle adjustment)

In [75]:
# Constants
num_cameras = 5
num_params_per_cam = 6
total_params = num_cameras * num_params_per_cam
n_particles = 60  # Number of particles
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.4} #{'c1': 2.05, 'c2': 2.05, 'w': 0.729} # PSO Hyperparameters

projMat = np.load('saved_cobotique/Extrinsics_optimized_15.npz')['arr_0']

init_params = []
for mat in projMat:
    rvec, _ = cv2.Rodrigues(mat[0:3,0:3])
    rvec = rvec.squeeze()
    params = [rvec[0], rvec[1], rvec[2], mat[0,3], mat[1,3], mat[2,3]]
    init_params.extend(params)

init_params = np.array(init_params, dtype=np.float64)
init_pos = np.tile(init_params, (n_particles,1))
print(init_params.shape)
print(init_pos.shape)

# Define parameter bounds
lower_bounds = [-2*np.pi, -2*np.pi, -2*np.pi, -3000, -3000, -3000]
upper_bounds = [2*np.pi, 2*np.pi, 2*np.pi, 3000, 3000, 3000]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))
print(param_bounds[0].shape)

print(np.all(param_bounds[0] <= init_pos[0]))
print(np.all(init_pos[0] <= param_bounds[1]))

(30,)
(60, 30)
(30,)
True
True


In [ ]:
def fun(params):
    errors = np.empty((n_particles,))
    params = np.array(params)

    for n in range(n_particles):
        
        projMat = np.empty((num_cameras+1, 3, 4))
        projMat[0] = np.hstack((np.eye((3)), np.zeros((3,1))))

        for cam_idx in range(num_cameras):
            cam_params = params[n][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            r1, r2, r3, t1, t2, t3 = cam_params
            rvec = np.array([r1, r2, r3])
            R, _ = cv2.Rodrigues(rvec)
            t = np.array([t1, t2, t3]).reshape((3,1))
            projMat[cam_idx+1] = np.hstack((R, t))  #[cam_idx+1]

        RMSE = {c:[] for c in range(6)}
        # Loop on stereo images checkberboard points
        for i in range(0, len(stereo_data['Camera']) - 1, 2):
            j = i+1 # stereo image is the next one

            pts1_im = stereo_data['Img_pts'][i].squeeze()
            pts2_im = stereo_data['Img_pts'][j].squeeze()

            c1 = int(stereo_data['Camera'][i]) - 1
            c2 = int(stereo_data['Camera'][j]) - 1

            """ t1 = float(stereo_data['Time'][i])
            t2 = float(stereo_data['Time'][j])
            assert abs(t1-t2) < time_threshold, f'Images are not matching: {t1:.4f} and {t2:.4f}' """

            undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2).T  # Shape (2, N)
            undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2).T  # Shape (2, N)

            # Perform triangulation
            pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], undist_pts1, undist_pts2)
            points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
            points_3d = points_3d.T  # Shape (N, 3)

            # Compute RMSE for both cameras
            rmse1 = compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])
            rmse2 = compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

            RMSE[c1].append(rmse1)
            RMSE[c2].append(rmse2)

        # Loop on tabletop reference points
        RMSE_tt = []
        for i in range(4):
            c1 = int(cams[i][-1]) - 1

            pts1_im = np.array(image_points[cams[i]], dtype=np.float32)
            undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2).T  # Shape (2, N)

            for j in range(i+1,4):
                c2 = int(cams[j][-1]) - 1

                pts2_im = np.array(image_points[cams[j]], dtype=np.float32)
                undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2).T  # Shape (2, N)

                # Perform triangulation
                pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], undist_pts1, undist_pts2)
                points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
                points_3d = points_3d.T  # Shape (N, 3)

                err_x = (304.8 - np.linalg.norm(points_3d[0] - points_3d[1]))**2
                err_y = (304.8 - np.linalg.norm(points_3d[0] - points_3d[2]))**2
                err_d = (431.0523 - np.linalg.norm(points_3d[1] - points_3d[2]))**2

                cost_dist = np.sqrt(np.mean([err_x, err_y, err_d]))

                rmse1_tt = compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])
                rmse2_tt = compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

                RMSE_tt.append(rmse1_tt)
                RMSE_tt.append(rmse2_tt)

        # Per-camera mean RMSE
        rmse_mean = []
        for _, rmse in RMSE.items():
            rmse_mean.append(np.mean(rmse))
        
        # global mean
        e = np.mean(rmse_mean) + np.mean(RMSE_tt) + cost_dist #+ np.max(RMSE)  + np.max(RMSE_tt)
        errors[n] = e   # OPTIONAL: add max error to cost function
        
    return errors

In [76]:
# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=total_params, options=options, bounds=param_bounds, init_pos=init_pos)
best_error, best_params = optimizer.optimize(fun, iters = 500)
print(f"Best Reprojection Error: {best_error} pixels")

# Extract optimized parameters for each camera
optimized_params = np.split(best_params, num_cameras)

plot_cost_history(cost_history=optimizer.cost_history)
plt.savefig('saved_cobotique/Figure_1_pso_cost_13.png')
plt.show()

2025-03-25 10:59:21,610 - pyswarms.single.global_best - INFO - Optimize for 500 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.4}
pyswarms.single.global_best:   0%|          |0/500

pyswarms.single.global_best: 100%|██████████|500/500, best_cost=8.01
2025-03-25 11:07:22,385 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8.014896389324036, best pos: [   -0.0255     2.0312     0.7895  -632.7989  -657.2509  2344.2767
     0.0184     1.3927     0.5038 -1435.8987  -220.5955  1939.3818
    -0.1865    -1.0159    -0.437    806.1666  -220.7843   169.7508
    -0.0661     1.0505     0.5993 -1873.4689  -752.8458  1182.9457
     0.1841    -0.4682     0.1174  -112.1726   255.2391   141.9347]


Best Reprojection Error: 8.014896389324036 pixels


In [77]:
extrinsics = []
extrinsics.append(np.hstack((np.eye((3)), np.zeros((3,1)))))

for cam_idx in range(num_cameras):
    r1, r2, r3, t1, t2, t3 = optimized_params[cam_idx]
    rvec = np.array([r1,r2,r3], dtype=np.float64)
    R, _ = cv2.Rodrigues(rvec)
    t = np.array([t1,t2,t3], dtype=np.float64).reshape((3,1))

    optimized_camera_matrix = np.hstack((R,t))
    extrinsics.append(optimized_camera_matrix)

for cam_idx, cam_mat in enumerate(extrinsics):
    print(f"Camera {cam_idx+1} Optimized Camera Matrix:\n", cam_mat)

np.savez('saved_cobotique/Extrinsics_optimized_16.npz', extrinsics[1:])

Camera 1 Optimized Camera Matrix:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
Camera 2 Optimized Camera Matrix:
 [[  -0.5715   -0.3143    0.758  -632.7989]
 [   0.2801    0.7935    0.5402 -657.2509]
 [  -0.7713    0.521    -0.3655 2344.2767]]
Camera 3 Optimized Camera Matrix:
 [[    0.0897    -0.3282     0.9403 -1435.8987]
 [    0.3494     0.8945     0.2789  -220.5955]
 [   -0.9327     0.3036     0.1949  1939.3818]]
Camera 4 Optimized Camera Matrix:
 [[   0.45      0.4362   -0.7793  806.1666]
 [  -0.2658    0.8985    0.3495 -220.7843]
 [   0.8526    0.0499    0.5202  169.7508]]
Camera 5 Optimized Camera Matrix:
 [[    0.3538    -0.4938     0.7944 -1873.4689]
 [    0.4325     0.8394     0.3292  -752.8458]
 [   -0.8294     0.2271     0.5105  1182.9457]]
Camera 6 Optimized Camera Matrix:
 [[   0.8861   -0.1544   -0.437  -112.1726]
 [   0.0701    0.9767   -0.2029  255.2391]
 [   0.4582    0.1492    0.8762  141.9347]]


## Cumulative error on matrices multiplication (Total transfo = I ?) 

In [78]:
T = []

for mat in extrinsics:
    M = np.vstack((mat, np.array([0,0,0,1])))
    T.append(M)

T_next = []
for i in range(len(T) - 1):
    T_next.append(np.linalg.inv(T[i]) @ T[i+1])

T5_calc = T_next[0] @ T_next[1] @ T_next[2] @ T_next[3]

print('T5 calc: \n', T5_calc)
print('T5: \n', T[4])

I_calc = T5_calc @ np.linalg.inv(T[4])
print('Cumulative error: \n', np.abs(I_calc - np.eye(4)))

T5 calc: 
 [[    0.3538    -0.4938     0.7944 -1873.4689]
 [    0.4325     0.8394     0.3292  -752.8458]
 [   -0.8294     0.2271     0.5105  1182.9457]
 [    0.         0.         0.         1.    ]]
T5: 
 [[    0.3538    -0.4938     0.7944 -1873.4689]
 [    0.4325     0.8394     0.3292  -752.8458]
 [   -0.8294     0.2271     0.5105  1182.9457]
 [    0.         0.         0.         1.    ]]
Cumulative error: 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## 2D visualization of original and projected points onto images

In [19]:
def draw_circles(img, pts_2d_original, pts_2d_projected, text=True):
    for p in range(len(pts_2d_original)):
        x_orig, y_orig = map(int, pts_2d_original[p])  # Original 2D point
        cv2.circle(img, (x_orig, y_orig), 8, (255, 0, 0), -1)

        if pts_2d_projected is not None:
            x_proj, y_proj = map(int, pts_2d_projected[p])  # Reprojected 3D point
            cv2.circle(img, (x_proj, y_proj), 8, (0, 255, 0), -1)
            cv2.line(img, (x_orig, y_orig), (x_proj, y_proj), (0, 255, 255), 2)

        if text:
            # Add labels
            cv2.putText(img, str(p + 1), (x_orig + 10, y_orig), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(img, str(p + 1), (x_proj + 10, y_proj), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0, 255, 0), 2, cv2.LINE_AA)
        
    return img

In [79]:
dir_images = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration_0703\corners_confirmed'

projMat = extrinsics

RMSE = []
rmse_per_cam = {'c1-c2':[], 'c1-c3':[], 'c1-c4':[], 'c1-c5':[], 'c1-c6':[],
                'c2-c3':[], 'c2-c4':[], 'c2-c5':[], 'c2-c6':[],
                'c3-c4':[], 'c3-c5':[], 'c3-c6':[], 'c4-c5':[], 'c4-c6':[], 'c5-c6':[]}

fig, axs = plt.subplots(6, 6, figsize=(20,14))
axs = axs.flat
k = 0

for i in range(0, len(stereo_data['Camera']) - 1, 8):
    j = i+1 # stereo image is the next one

    pts1_im = stereo_data['Img_pts'][i].squeeze()
    pts2_im = stereo_data['Img_pts'][j].squeeze()

    c1 = int(stereo_data['Camera'][i]) - 1
    c2 = int(stereo_data['Camera'][j]) - 1

    t1 = float(stereo_data['Time'][i])
    t2 = float(stereo_data['Time'][j])
    assert abs(t1-t2) < time_threshold, f'Images are not matching: {t1:.4f} and {t2:.4f}'

    undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2).T  # Shape (2, N)
    undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2).T  # Shape (2, N)

    # Perform triangulation
    pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], undist_pts1, undist_pts2)

    # Convert from homogeneous coordinates
    points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
    points_3d = points_3d.T  # Shape (N, 3)

    img_path1 = os.path.join(dir_images, f'c{c1+1}', stereo_data['Name'][i])
    img_path2 = os.path.join(dir_images, f'c{c2+1}', stereo_data['Name'][j])

    # Visualize original and reprojected points (image 1)
    img = cv2.imread(img_path1)

    pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c1][0:3,0:3], projMat[c1][0:3,3], K[c1], D[c1])
    pts_2d_projected = pts_2d_projected.squeeze()
    rmse1 = compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])

    if i % 8 == 0:
        img = draw_circles(img, pts1_im, pts_2d_projected)
        axs[k].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k].axis('off')
        axs[k].set_title(f'{stereo_data['Name'][i]} RMSE = {rmse1:.2f}', fontsize=8)

    # Visualize original and reprojected points (image 2)
    img = cv2.imread(img_path2)
    
    pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c2][0:3,0:3], projMat[c2][0:3,3], K[c2], D[c2])
    pts_2d_projected = pts_2d_projected.squeeze()
    rmse2 = compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

    if i % 8 == 0:
        img = draw_circles(img, pts2_im, pts_2d_projected)
        axs[k+1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k+1].axis('off')
        axs[k+1].set_title(f'{stereo_data['Name'][j]} RMSE = {rmse2:.2f}', fontsize=8)
        k += 2

    rmse_per_cam[f'c{c1+1}-c{c2+1}'] += [rmse1, rmse2]

fig.suptitle("Original 2D Points (Blue) vs. Projected 3D Points (Green)")
fig.tight_layout()
plt.savefig('Figure_reprojection_23a.png')
plt.show()

for key, vals in rmse_per_cam.items():
    print(key, np.mean(vals))

c1-c2 nan
c1-c3 nan
c1-c4 4.761566
c1-c5 14.065009
c1-c6 0.17111018
c2-c3 1.5695016
c2-c4 nan
c2-c5 2.9300756
c2-c6 nan
c3-c4 nan
c3-c5 0.6328215
c3-c6 nan
c4-c5 nan
c4-c6 21.653002
c5-c6 nan


## 3D visualization of cameras in space

In [51]:
def plot_frame(ax, T, label="Frame", length=100.0):
    """Plots a 3D reference frame with arrows for X, Y, and Z axes."""
    origin = T[:3, 3]  # Extract translation (position)
    R = T[:3, :3]      # Extract rotation matrix (orientation)

    # Define the x, y, z unit vectors from the rotation matrix
    x_axis = R[:, 0] * length  # X-direction
    y_axis = R[:, 1] * length  # Y-direction
    z_axis = R[:, 2] * length  # Z-direction

    # Plot arrows for the axes
    ax.quiver(*origin, *x_axis, color='r', arrow_length_ratio=0.3)  # X-axis (Red)
    ax.quiver(*origin, *y_axis, color='g', arrow_length_ratio=0.3)  # Y-axis (Green)
    ax.quiver(*origin, *z_axis, color='b', arrow_length_ratio=0.3)  # Z-axis (Blue)

    # Label the frame
    ax.text(*origin, label, fontsize=12, color='black')

    return

In [52]:
# Plot frames
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, T in enumerate(extrinsics):
    T = np.vstack((T, np.array([0,0,0,1])))
    plot_frame(ax, T, label=f'Camera {i+1}')

# Labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("3D Reference Frames")
plt.show()

### Triangulation of reference points

In [80]:
# Load points from JSON file
with open("saved_cobotique/clicked_points_adjusted.json", "r") as file:
    image_points = json.load(file)

fig, axs = plt.subplots(5, 6, figsize=(16,12))
axs = axs.flatten()
k = 0

projMat = extrinsics

ref_points = []


for i in range(6):
    c1 = int(cams[i][-1]) - 1

    for j in range(i+1,6):
        c2 = int(cams[j][-1]) - 1
        print(c1, c2)
        
        pts1_im = np.array(image_points[cams[i]], dtype=np.float32)
        pts2_im = np.array(image_points[cams[j]], dtype=np.float32)

        img_path1 = img_paths[i]
        img_path2 = img_paths[j]

        undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2).T  # Shape (2, N)
        undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2).T  # Shape (2, N)

        # Perform triangulation
        pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], undist_pts1, undist_pts2)

        # Convert from homogeneous coordinates
        points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
        points_3d = points_3d.T  # Shape (N, 3)
        print(points_3d)

        # Visualize original and reprojected points (image 1)
        img = cv2.imread(img_path1)

        pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c1][0:3,0:3], projMat[c1][0:3,3], K[c1], D[c1])
        pts_2d_projected = pts_2d_projected.squeeze()
        rmse1 = compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])

        print(rmse1)

        img = draw_circles(img, pts1_im, pts_2d_projected)
        axs[k].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k].axis('off')
        axs[k].set_title(f'{stereo_data['Name'][i]} RMSE = {rmse1:.2f}', fontsize=8)

        # Visualize original and reprojected points (image 2)
        img = cv2.imread(img_path2)
        
        pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c2][0:3,0:3], projMat[c2][0:3,3], K[c2], D[c2])
        pts_2d_projected = pts_2d_projected.squeeze()
        rmse2 = compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

        print(rmse2)

        if rmse1 < 1 and rmse2 < 1:
            ref_points.append(points_3d)

        img = draw_circles(img, pts2_im, pts_2d_projected)
        axs[k+1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k+1].axis('off')
        axs[k+1].set_title(f'{stereo_data['Name'][j]} RMSE = {rmse2:.2f}', fontsize=8)

        k += 2

fig.suptitle("Original 2D Points (Blue) vs. Projected 3D Points (Green)")
fig.tight_layout()
plt.savefig('Figure_reprojection_20b.png')
plt.show()
plt.close()


0 1
[[ 438.9573  612.0837 1547.7601]
 [ 597.9066  711.9742 1308.5292]
 [ 699.2227  561.614  1688.9116]]
5.492077
5.1638227
0 2
[[ 438.793   613.6914 1567.5546]
 [ 594.5112  718.7676 1320.204 ]
 [ 694.4538  558.0953 1703.2388]]
3.368347
2.668509
0 3
[[ 438.4645  564.0198 1502.3374]
 [ 584.0068  668.7175 1283.5745]
 [ 683.8554  496.4674 1629.6431]]
14.8665695
13.897979
0 4
[[ 443.907   621.881  1555.2384]
 [ 600.2332  726.2197 1307.9934]
 [ 708.6593  574.9991 1718.9465]]
8.828814
7.624548
0 5
[[ 468.4442  659.6081 1675.1582]
 [ 619.8766  750.5546 1393.7847]
 [ 751.7582  609.4136 1855.8385]]
1.7495277
1.6301049
1 2
[[ 484.8057  562.0983 1552.0518]
 [ 635.1198  683.4676 1305.4451]
 [ 742.7792  505.2459 1682.3501]]
0.9822641
0.8384604
1 3
[[ 484.5439  564.8813 1546.3999]
 [ 635.4725  679.1663 1308.2439]
 [ 756.7451  488.5258 1682.8137]]
1.4410608
1.6889228
1 4
[[ 433.8782  632.5175 1549.067 ]
 [ 597.9603  748.4139 1293.452 ]
 [ 729.8957  549.9414 1690.686 ]]
4.4517183
4.741482
1 5
[[ 441.68

### 3D reference points on tabletop as global reference frame

In [26]:
ref_points = np.array(ref_points)
ref_points_mean = np.mean(ref_points, axis=0)
print(ref_points_mean)

t = ref_points_mean[0].reshape((3,1))
x = ref_points_mean[1] - ref_points_mean[0] / np.linalg.norm(ref_points_mean[1] - ref_points_mean[0])
y = ref_points_mean[2] - ref_points_mean[0] / np.linalg.norm(ref_points_mean[2] - ref_points_mean[0])

z = np.cross(x, y)

x = x.reshape((3,1))
y = y.reshape((3,1))
z = z.reshape((3,1))

R_table = np.hstack((x,y,z))
T_table = np.vstack((np.hstack((R, t)), np.array([0,0,0,1])))

tt_T_world = np.linalg.inv(T_table)
print(tt_T_world)

[[ 491.8386  686.3479 1740.49  ]
 [ 666.5218  804.0645 1488.0469]
 [ 777.1145  626.6667 1908.1335]]
[[    0.8824     0.0817     0.4633 -1296.4921]
 [   -0.1719     0.9727     0.1558  -854.2322]
 [   -0.4379    -0.2171     0.8724 -1153.9842]
 [    0.         0.         0.         1.    ]]


In [27]:
# Plot frames
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, T in enumerate(extrinsics):
    T = np.vstack((T, np.array([0,0,0,1])))
    T = tt_T_world @ T
    plot_frame(ax, T, label=f'Camera {i+1}')

# Labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("3D Reference Frames")

plt.show()